
<h1 id="Yield-Management">Yield Management<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Yield-Management">¶</a></h1><h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Objective-and-Prerequisites">¶</a></h2><p>This model is an example of a Yield Management problem. 
Yield management is a variable pricing strategy, based on understanding, anticipating, and influencing consumer behavior in order to maximize revenue or profits from a fixed, time-limited resource (such as airline seats).  This Yield Management Problem is formulated as a three-period stochastic programming problem using the Gurobi Python API and solved with the Gurobi Optimizer.</p>
<p>This model is example 24 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 282-284 and 337-340.</p>
<p>This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.</p>
<p><strong>Download the Repository</strong> <br/> 
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-MUI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Yield_Management_COM_EVAL_GitHub&amp;utm_term=Yield%20Management&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Yield_Management_COM_EVAL_GitHub&amp;utm_term=Yield%20Management&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>



<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Problem-Description">¶</a></h2><p>An airline is selling tickets for flights to a particular destination. The flights will depart in three weeks and require up to six planes, each costing £50 000 to hire. Each plane has the following:</p>
<ul>
<li>37 First Class seats</li>
<li>38 Business Class seats</li>
<li>47 Economy Class seats.</li>
</ul>
<p>The airline needs to decide on an initial price for each of these seats, and will then have the opportunity to update the price after one week and two weeks. Once a customer has purchased a ticket, there is no cancellation option. For administrative simplicity, three price level options are possible in each class (one of which must be chosen). The same option need not be chosen for each class. These are given in the following table for the current period (period 1) and two future periods.</p>
<p><img alt="optionsClass" src="./yield_management_files/optionsClass.PNG"/></p>
<p>Demand is uncertain but will be affected by price. Demand forecasts have been made according to a probability distribution that divides the demand levels into three scenarios for each period. The probabilities of the three scenarios
in each period are as follows:</p>
<p><img alt="scenariosProb" src="./yield_management_files/scenariosProb.PNG"/></p>
<p>The forecasted demand levels are shown in the following table:</p>
<p><img alt="forecastDemand" src="./yield_management_files/forecastDemand.PNG"/></p>
<p>The goal is to determine the price levels for the current period, how many seats to sell in each class, the provisional number of planes to book and provisional price levels and seats to sell in future periods in order to maximize
expected yield. We should be able to meet commitments under all possible combinations of scenarios. With hindsight (i.e. not known until the beginning of the next period), it turned out that demand in each period (depending on the price level you chose) was as shown in following table.</p>
<p><img alt="actualDemand" src="./yield_management_files/actualDemand.PNG"/></p>
<p>We use the actual demand that resulted from the prices we set in period 1 to rerun the model at the beginning of period 2 to set price levels for period 2 and provisional price levels for period 3. We repeat this procedure with a rerun at the beginning of period 3.</p>



<hr/>
<h2 id="Model-Formulation">Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Model-Formulation">¶</a></h2><p>The Yield Management problem is formulated as a three-period stochastic programming problem. Solving this model for the first time gives recommendations for price levels and sales of week 1 and recommends price levels and sales for subsequent weeks under all possible scenarios. The probabilities of these scenarios will be taken into account in order to maximize expected yield. A week later the model will be rerun, taking into account the committed sales and revenue in the first week, to redetermine recommended prices and sales for the second week (i.e. with ‘recourse’) and the third week under all possible scenarios. The procedure will be repeated again a week later.</p>
<h3 id="Sets-and-indices">Sets and indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Sets-and-indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 9.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1007.44em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-4" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-5" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-6" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-7" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-8" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-9" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>∈</mo><mtext>Scenarios</mtext></math></span></span><script id="MathJax-Element-1" type="math/tex">i,j,k \in \text{Scenarios}</script>: Indices and set of scenarios.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-10" style="width: 6.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1005.24em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-11"><span class="mi" id="MathJax-Span-12" style="font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-13" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-14" style="font-family: MathJax_Main; padding-left: 0.301em;">Options</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></math></span></span><script id="MathJax-Element-2" type="math/tex">h \in \text{Options}</script>: Index and set of price options.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-15" style="width: 4.824em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.991em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1003.93em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-16"><span class="mi" id="MathJax-Span-17" style="font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-18" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-19" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.075em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi><mo>∈</mo><mtext>Class</mtext></math></span></span><script id="MathJax-Element-3" type="math/tex">c \in \text{Class}</script>: Index and set of seats categories.</p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;price1&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-20" style="width: 7.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.193em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.19em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-21"><span class="msubsup" id="MathJax-Span-22"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.56em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-23" style="font-family: MathJax_Main;">price1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.622em;"><span class="texatom" id="MathJax-Span-24"><span class="mrow" id="MathJax-Span-25"><span class="mi" id="MathJax-Span-26" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-27" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-28" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-29" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-30" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-31"><span class="mrow" id="MathJax-Span-32"><span class="mi" id="MathJax-Span-33" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-34" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>price1</mtext><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-4" type="math/tex">\text{price1}_{c,h} \in \mathbb{R}^+</script>: Price of option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-35" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-36"><span class="mi" id="MathJax-Span-37" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-5" type="math/tex">h</script> chosen for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-38" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-39"><span class="mi" id="MathJax-Span-40" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-6" type="math/tex">c</script> in week 1.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;price2&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-41" style="width: 8.039em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.67em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.67em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-42"><span class="msubsup" id="MathJax-Span-43"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.56em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-44" style="font-family: MathJax_Main;">price2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.622em;"><span class="texatom" id="MathJax-Span-45"><span class="mrow" id="MathJax-Span-46"><span class="mi" id="MathJax-Span-47" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-48" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-49" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-50" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-51" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-52" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-53" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-54"><span class="mrow" id="MathJax-Span-55"><span class="mi" id="MathJax-Span-56" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-57" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>price2</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-7" type="math/tex">\text{price2}_{i,c,h} \in \mathbb{R}^+</script>: Price of option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-58" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-59"><span class="mi" id="MathJax-Span-60" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-8" type="math/tex">h</script> chosen for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-61" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-62"><span class="mi" id="MathJax-Span-63" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-9" type="math/tex">c</script> in week 2 as a result of scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-64" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-65"><span class="mi" id="MathJax-Span-66" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-10" type="math/tex">i</script> in week 1.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;price3&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-67" style="width: 8.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.146em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.15em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-68"><span class="msubsup" id="MathJax-Span-69"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.56em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-70" style="font-family: MathJax_Main;">price3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.622em;"><span class="texatom" id="MathJax-Span-71"><span class="mrow" id="MathJax-Span-72"><span class="mi" id="MathJax-Span-73" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-74" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-75" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-76" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-77" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-78" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-79" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-80" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-81" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-82"><span class="mrow" id="MathJax-Span-83"><span class="mi" id="MathJax-Span-84" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-85" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>price3</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-11" type="math/tex">\text{price3}_{i,j,c,h} \in \mathbb{R}^+</script>: Price of option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-86" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-87"><span class="mi" id="MathJax-Span-88" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-12" type="math/tex">h</script> chosen for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-89" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-90"><span class="mi" id="MathJax-Span-91" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-13" type="math/tex">c</script> in week 3 as a result of scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-92" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-93"><span class="mi" id="MathJax-Span-94" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-14" type="math/tex">i</script> in week 1, and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-95" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-96"><span class="mi" id="MathJax-Span-97" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-15" type="math/tex">j</script> in week 2.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;forecast1&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-98" style="width: 9.527em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.92em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-99"><span class="msubsup" id="MathJax-Span-100"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.81em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-101" style="font-family: MathJax_Main;">forecast1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.872em;"><span class="texatom" id="MathJax-Span-102"><span class="mrow" id="MathJax-Span-103"><span class="mi" id="MathJax-Span-104" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-105" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-106" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-107" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-108" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-109" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-110" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-111"><span class="mrow" id="MathJax-Span-112"><span class="mi" id="MathJax-Span-113" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-114" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>forecast1</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-16" type="math/tex">\text{forecast1}_{i,c,h} \in \mathbb{R}^+</script>: Forecast demand in week 1 for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-115" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-116"><span class="mi" id="MathJax-Span-117" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-17" type="math/tex">c</script> under price option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-118" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-119"><span class="mi" id="MathJax-Span-120" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-18" type="math/tex">h</script> and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-121" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-122"><span class="mi" id="MathJax-Span-123" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-19" type="math/tex">i</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;forecast2&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-124" style="width: 10.122em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.396em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.4em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-125"><span class="msubsup" id="MathJax-Span-126"><span style="display: inline-block; position: relative; width: 5.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.81em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-127" style="font-family: MathJax_Main;">forecast2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.872em;"><span class="texatom" id="MathJax-Span-128"><span class="mrow" id="MathJax-Span-129"><span class="mi" id="MathJax-Span-130" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-131" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-132" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-133" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-134" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-135" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-136" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-137" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-138" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-139"><span class="mrow" id="MathJax-Span-140"><span class="mi" id="MathJax-Span-141" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-142" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>forecast2</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-20" type="math/tex">\text{forecast2}_{i,j,c,h} \in \mathbb{R}^+</script>: Forecast demand in week 2 for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-143" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-144"><span class="mi" id="MathJax-Span-145" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-21" type="math/tex">c</script> under price option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-146" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-147"><span class="mi" id="MathJax-Span-148" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-22" type="math/tex">h</script> if scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-149" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-150"><span class="mi" id="MathJax-Span-151" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-23" type="math/tex">i</script> holds in week 1, and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-152" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-153"><span class="mi" id="MathJax-Span-154" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-24" type="math/tex">j</script> in week 2.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;forecast3&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-155" style="width: 10.836em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.991em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.99em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-156"><span class="msubsup" id="MathJax-Span-157"><span style="display: inline-block; position: relative; width: 6.372em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.81em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-158" style="font-family: MathJax_Main;">forecast3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.872em;"><span class="texatom" id="MathJax-Span-159"><span class="mrow" id="MathJax-Span-160"><span class="mi" id="MathJax-Span-161" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-162" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-163" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-164" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-165" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-166" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-167" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-168" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-169" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-170" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-171" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-172"><span class="mrow" id="MathJax-Span-173"><span class="mi" id="MathJax-Span-174" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-175" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>forecast3</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-25" type="math/tex">\text{forecast3}_{i,j,k,c,h} \in \mathbb{R}^+</script>: Forecast demand in week 3 for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-176" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-177"><span class="mi" id="MathJax-Span-178" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-26" type="math/tex">c</script> under price option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-179" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-180"><span class="mi" id="MathJax-Span-181" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-27" type="math/tex">h</script> if  scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-182" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-183"><span class="mi" id="MathJax-Span-184" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-28" type="math/tex">i</script> holds in week 1, scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-185" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-186"><span class="mi" id="MathJax-Span-187" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-29" type="math/tex">j</script> in week 2, and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-188" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-189"><span class="mi" id="MathJax-Span-190" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-30" type="math/tex">k</script> in week 3.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-31-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-191" style="width: 6.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.48em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-192"><span class="msubsup" id="MathJax-Span-193"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-194" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-195" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-196" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-197" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-198" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-199" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-200" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-201" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>prob</mtext><mi>i</mi></msub><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-31" type="math/tex">\text{prob}_i \in [0,1]</script>: Probability of scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-32-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-202" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-203"><span class="mi" id="MathJax-Span-204" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-32" type="math/tex">i</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;cap&lt;/mtext&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-33-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-205" style="width: 4.646em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.87em, 2.92em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-206"><span class="msubsup" id="MathJax-Span-207"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-208" style="font-family: MathJax_Main;">cap</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 1.491em;"><span class="mi" id="MathJax-Span-209" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-210" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-211" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-212"><span class="mi" id="MathJax-Span-213" style="font-family: MathJax_AMS;">N</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>cap</mtext><mi>c</mi></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow></math></span></span><script id="MathJax-Element-33" type="math/tex">\text{cap}_c \in \mathbb{N}</script>: Capacity per plane for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-34-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-214" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-215"><span class="mi" id="MathJax-Span-216" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-34" type="math/tex">c</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;cost&lt;/mtext&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-35-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-217" style="width: 5.241em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.348em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1004.35em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-218"><span class="mtext" id="MathJax-Span-219" style="font-family: MathJax_Main;">cost</span><span class="mo" id="MathJax-Span-220" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-221" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-222"><span class="mrow" id="MathJax-Span-223"><span class="mi" id="MathJax-Span-224" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-225" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>cost</mtext><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-35" type="math/tex">\text{cost} \in \mathbb{R}^+</script>: Cost to hire a plane.</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-36-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-226" style="width: 6.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-227"><span class="mi" id="MathJax-Span-228" style="font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-229"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-230" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-231"><span class="mrow" id="MathJax-Span-232"><span class="mi" id="MathJax-Span-233" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-234" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-235" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-236" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-237" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-238" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-239" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-240" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-241" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-36" type="math/tex">p1_{c,h} \in \{0, 1\}</script>: This binary variable is equal to one if price of option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-37-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-242" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-243"><span class="mi" id="MathJax-Span-244" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-37" type="math/tex">h</script> is chosen for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-38-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-245" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-246"><span class="mi" id="MathJax-Span-247" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-38" type="math/tex">c</script> in week 1.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-39-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-248" style="width: 7.384em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.134em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.07em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-249"><span class="mi" id="MathJax-Span-250" style="font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-251"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-252" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-253"><span class="mrow" id="MathJax-Span-254"><span class="mi" id="MathJax-Span-255" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-256" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-257" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-258" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-259" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-260" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-261" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-262" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-263" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-264" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-265" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-39" type="math/tex">p2_{i,c,h} \in \{0, 1\}</script>: This binary variable is equal to one if price of option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-40-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-266" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-267"><span class="mi" id="MathJax-Span-268" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-40" type="math/tex">h</script> is chosen for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-41-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-269" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-270"><span class="mi" id="MathJax-Span-271" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-41" type="math/tex">c</script> in week 2 as a result of scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-42-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-272" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-273"><span class="mi" id="MathJax-Span-274" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-42" type="math/tex">i</script> in week 1.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-43-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-275" style="width: 8.039em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.67em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.61em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-276"><span class="mi" id="MathJax-Span-277" style="font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-278"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-279" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-280"><span class="mrow" id="MathJax-Span-281"><span class="mi" id="MathJax-Span-282" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-283" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-284" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-285" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-286" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-287" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-288" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-289" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-290" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-291" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-292" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-293" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-294" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-43" type="math/tex">p3_{i,j,c,h} \in \{0, 1\}</script>: This binary variable is equal to one if price of option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-44-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-295" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-296"><span class="mi" id="MathJax-Span-297" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-44" type="math/tex">h</script> is chosen for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-45-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-298" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-299"><span class="mi" id="MathJax-Span-300" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-45" type="math/tex">c</script> in week 3 as a result of scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-46-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-301" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-302"><span class="mi" id="MathJax-Span-303" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-46" type="math/tex">i</script> in week 1 and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-47-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-304" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-305"><span class="mi" id="MathJax-Span-306" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-47" type="math/tex">j</script> in week 2.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-48-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-307" style="width: 6.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.003em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-308"><span class="mi" id="MathJax-Span-309" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-310"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-311" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-312"><span class="mrow" id="MathJax-Span-313"><span class="mi" id="MathJax-Span-314" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-315" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-316" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-317" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-318" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-319" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-320" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-321"><span class="mrow" id="MathJax-Span-322"><span class="mi" id="MathJax-Span-323" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-324" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-48" type="math/tex">s1_{i,c,h} \in \mathbb{R}^+</script>: Number of tickets to be sold in week 1 for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-49-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-325" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-326"><span class="mi" id="MathJax-Span-327" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-49" type="math/tex">c</script> under price option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-50-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-328" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-329"><span class="mi" id="MathJax-Span-330" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-50" type="math/tex">h</script> and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-51-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-331" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-332"><span class="mi" id="MathJax-Span-333" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-51" type="math/tex">i</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-52-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-334" style="width: 6.67em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.54em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-335"><span class="mi" id="MathJax-Span-336" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-337"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-338" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-339"><span class="mrow" id="MathJax-Span-340"><span class="mi" id="MathJax-Span-341" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-342" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-343" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-344" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-345" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-346" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-347" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-348" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-349" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-350"><span class="mrow" id="MathJax-Span-351"><span class="mi" id="MathJax-Span-352" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-353" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-52" type="math/tex">s2_{i,j,c,h} \in \mathbb{R}^+</script>: Number of tickets to be sold in week 2 for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-53-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-354" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-355"><span class="mi" id="MathJax-Span-356" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-53" type="math/tex">c</script> under price option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-54-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-357" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-358"><span class="mi" id="MathJax-Span-359" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-54" type="math/tex">h</script> if scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-55-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-360" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-361"><span class="mi" id="MathJax-Span-362" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-55" type="math/tex">i</script> holds in week 1, and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-56-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-363" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-364"><span class="mi" id="MathJax-Span-365" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-56" type="math/tex">j</script> in week 2.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-57-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-366" style="width: 7.324em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.074em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.07em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-367"><span class="mi" id="MathJax-Span-368" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-369"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-370" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-371"><span class="mrow" id="MathJax-Span-372"><span class="mi" id="MathJax-Span-373" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-374" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-375" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-376" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-377" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-378" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-379" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-380" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-381" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-382" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-383" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-384"><span class="mrow" id="MathJax-Span-385"><span class="mi" id="MathJax-Span-386" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-387" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-57" type="math/tex">s3_{i,j,k,c,h} \in \mathbb{R}^+</script>: Number of tickets to be sold in week 3 for class <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-58-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-388" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-389"><span class="mi" id="MathJax-Span-390" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-58" type="math/tex">c</script> under price option <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-59-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-391" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-392"><span class="mi" id="MathJax-Span-393" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-59" type="math/tex">h</script> if  scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-60-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-394" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-395"><span class="mi" id="MathJax-Span-396" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-60" type="math/tex">i</script> in week 1, scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-61-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-397" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-398"><span class="mi" id="MathJax-Span-399" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-61" type="math/tex">j</script> in week 2, and scenario <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-62-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-400" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-401"><span class="mi" id="MathJax-Span-402" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-62" type="math/tex">k</script> in week 3.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-63-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-403" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1002.62em, 2.741em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-404"><span class="mi" id="MathJax-Span-405" style="font-family: MathJax_Math-italic;">n</span><span class="mo" id="MathJax-Span-406" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-407" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-408"><span class="mi" id="MathJax-Span-409" style="font-family: MathJax_AMS;">N</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>n</mi><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow></math></span></span><script id="MathJax-Element-63" type="math/tex">n \in \mathbb{N}</script>: Number of planes to fly.</p>
<h3 id="Objective-function">Objective function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Objective-function">¶</a></h3><p><strong>Profit</strong>: Maximize expected profit.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtext&gt;Maximize&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mtext&gt;profit&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;price1&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-64-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-410" style="width: 39.943em; display: inline-block;"><span style="display: inline-block; position: relative; width: 33.277em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1033.22em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-411"><span class="mtext" id="MathJax-Span-412" style="font-family: MathJax_Main;">Maximize</span><span class="mspace" id="MathJax-Span-413" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mtext" id="MathJax-Span-414" style="font-family: MathJax_Main;">profit</span><span class="mo" id="MathJax-Span-415" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-416" style="font-family: MathJax_Main; padding-left: 0.301em;">(</span><span class="munderover" id="MathJax-Span-417"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.074em;"><span class="mo" id="MathJax-Span-418" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.57em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-419"><span class="mrow" id="MathJax-Span-420"><span class="mi" id="MathJax-Span-421" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-422" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-423" style="font-size: 70.7%; font-family: MathJax_Main;">Scenarios</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-424" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.479em;"><span class="mo" id="MathJax-Span-425" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.38em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-426"><span class="mrow" id="MathJax-Span-427"><span class="mi" id="MathJax-Span-428" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-429" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-430" style="font-size: 70.7%; font-family: MathJax_Main;">Class</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-431" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-432" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-433"><span class="mrow" id="MathJax-Span-434"><span class="mi" id="MathJax-Span-435" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-436" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-437" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-438" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-439"><span class="msubsup" id="MathJax-Span-440"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-441" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-442" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-443" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-444" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.56em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-445" style="font-family: MathJax_Main;">price1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.622em;"><span class="texatom" id="MathJax-Span-446"><span class="mrow" id="MathJax-Span-447"><span class="mi" id="MathJax-Span-448" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-449" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-450" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-451" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-452" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">p</span><span class="msubsup" id="MathJax-Span-453"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-454" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-455"><span class="mrow" id="MathJax-Span-456"><span class="mi" id="MathJax-Span-457" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-458" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-459" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-460" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-461" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">s</span><span class="msubsup" id="MathJax-Span-462"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-463" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-464"><span class="mrow" id="MathJax-Span-465"><span class="mi" id="MathJax-Span-466" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-467" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-468" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-469" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-470" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-471" style="font-family: MathJax_Main;">+</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtext>Maximize</mtext><mspace width="1em"></mspace><mtext>profit</mtext><mo>=</mo><mo stretchy="false">(</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mtext>Scenarios</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>∈</mo><mtext>Class</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>prob</mtext><mi>i</mi></msub><mo>∗</mo><msub><mtext>price1</mtext><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>p</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>s</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub></mrow><mo>+</mo></math></span></span></div><script id="MathJax-Element-64" type="math/tex; mode=display">
\text{Maximize} \quad \text{profit} = (
\sum_{i \in \text{Scenarios}} \sum_{c \in \text{Class}} 
\sum_{h \in \text{Options}}{\text{prob}_i * \text{price1}_{c,h} * p1_{c,h} * s1_{i,c,h}  }  +
</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;price2&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-65-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-472" style="width: 38.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 31.967em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1031.91em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-473"><span class="munderover" id="MathJax-Span-474"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.074em;"><span class="mo" id="MathJax-Span-475" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.57em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-476"><span class="mrow" id="MathJax-Span-477"><span class="mi" id="MathJax-Span-478" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-479" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-480" style="font-size: 70.7%; font-family: MathJax_Main;">Scenarios</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-481" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 3.634em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.074em;"><span class="mo" id="MathJax-Span-482" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.63em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-483"><span class="mrow" id="MathJax-Span-484"><span class="mi" id="MathJax-Span-485" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-486" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-487" style="font-size: 70.7%; font-family: MathJax_Main;">Scenarios</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-488" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.479em;"><span class="mo" id="MathJax-Span-489" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.38em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-490"><span class="mrow" id="MathJax-Span-491"><span class="mi" id="MathJax-Span-492" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-493" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-494" style="font-size: 70.7%; font-family: MathJax_Main;">Class</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-495" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-496" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-497"><span class="mrow" id="MathJax-Span-498"><span class="mi" id="MathJax-Span-499" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-500" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-501" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-502" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-503"><span class="msubsup" id="MathJax-Span-504"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-505" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-506" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-507" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-508" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-509" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-510" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-511" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-512" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.56em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-513" style="font-family: MathJax_Main;">price2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.622em;"><span class="texatom" id="MathJax-Span-514"><span class="mrow" id="MathJax-Span-515"><span class="mi" id="MathJax-Span-516" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-517" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-518" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-519" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-520" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">p</span><span class="msubsup" id="MathJax-Span-521"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-522" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-523"><span class="mrow" id="MathJax-Span-524"><span class="mi" id="MathJax-Span-525" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-526" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-527" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-528" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-529" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-530" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-531" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">s</span><span class="msubsup" id="MathJax-Span-532"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-533" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-534"><span class="mrow" id="MathJax-Span-535"><span class="mi" id="MathJax-Span-536" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-537" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-538" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-539" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-540" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-541" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-542" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-543" style="font-family: MathJax_Main;">+</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mtext>Scenarios</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mtext>Scenarios</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>∈</mo><mtext>Class</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>prob</mtext><mi>i</mi></msub><mo>∗</mo><msub><mtext>prob</mtext><mi>j</mi></msub><mo>∗</mo><msub><mtext>price2</mtext><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>p</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>s</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub></mrow><mo>+</mo></math></span></span></div><script id="MathJax-Element-65" type="math/tex; mode=display">
\sum_{i \in \text{Scenarios}} \sum_{j \in \text{Scenarios}} \sum_{c \in \text{Class}} \sum_{h \in \text{Options}}
{\text{prob}_i *\text{prob}_j * \text{price2}_{c,h} * p2_{i,c,h} * s2_{i,j,c,h} } +
</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;prob&lt;/mtext&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;price3&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mtext&gt;cost&lt;/mtext&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-66-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-544" style="width: 53.455em; display: inline-block;"><span style="display: inline-block; position: relative; width: 44.527em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1044.53em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-545"><span class="munderover" id="MathJax-Span-546"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.074em;"><span class="mo" id="MathJax-Span-547" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.57em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-548"><span class="mrow" id="MathJax-Span-549"><span class="mi" id="MathJax-Span-550" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-551" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-552" style="font-size: 70.7%; font-family: MathJax_Main;">Scenarios</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-553" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 3.634em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.074em;"><span class="mo" id="MathJax-Span-554" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.63em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-555"><span class="mrow" id="MathJax-Span-556"><span class="mi" id="MathJax-Span-557" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-558" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-559" style="font-size: 70.7%; font-family: MathJax_Main;">Scenarios</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-560" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 3.693em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.134em;"><span class="mo" id="MathJax-Span-561" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.69em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-562"><span class="mrow" id="MathJax-Span-563"><span class="mi" id="MathJax-Span-564" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-565" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-566" style="font-size: 70.7%; font-family: MathJax_Main;">Scenarios</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-567" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.479em;"><span class="mo" id="MathJax-Span-568" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.38em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-569"><span class="mrow" id="MathJax-Span-570"><span class="mi" id="MathJax-Span-571" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-572" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-573" style="font-size: 70.7%; font-family: MathJax_Main;">Class</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-574" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-575" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-576"><span class="mrow" id="MathJax-Span-577"><span class="mi" id="MathJax-Span-578" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-579" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-580" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-581" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-582"><span class="msubsup" id="MathJax-Span-583"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-584" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-585" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-586" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-587" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-588" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-589" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-590" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-591" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-592" style="font-family: MathJax_Main;">prob</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="mi" id="MathJax-Span-593" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-594" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-595" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.56em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-596" style="font-family: MathJax_Main;">price3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.622em;"><span class="texatom" id="MathJax-Span-597"><span class="mrow" id="MathJax-Span-598"><span class="mi" id="MathJax-Span-599" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-600" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-601" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-602" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-603" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">p</span><span class="msubsup" id="MathJax-Span-604"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-605" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-606"><span class="mrow" id="MathJax-Span-607"><span class="mi" id="MathJax-Span-608" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-609" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-610" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-611" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-612" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-613" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-614" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-615" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-616" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">s</span><span class="msubsup" id="MathJax-Span-617"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-618" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-619"><span class="mrow" id="MathJax-Span-620"><span class="mi" id="MathJax-Span-621" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-622" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-623" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-624" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-625" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-626" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-627" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-628" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-629" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-630" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-631" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mtext" id="MathJax-Span-632" style="font-family: MathJax_Main; padding-left: 0.241em;">cost</span><span class="mo" id="MathJax-Span-633" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-634" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">n</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mtext>Scenarios</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mtext>Scenarios</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mtext>Scenarios</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>∈</mo><mtext>Class</mtext></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>prob</mtext><mi>i</mi></msub><mo>∗</mo><msub><mtext>prob</mtext><mi>j</mi></msub><mo>∗</mo><msub><mtext>prob</mtext><mi>k</mi></msub><mo>∗</mo><msub><mtext>price3</mtext><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>p</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>s</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub></mrow><mo stretchy="false">)</mo><mo>−</mo><mtext>cost</mtext><mo>∗</mo><mi>n</mi></math></span></span></div><script id="MathJax-Element-66" type="math/tex; mode=display">
\sum_{i \in \text{Scenarios}} \sum_{j \in \text{Scenarios}} \sum_{k \in \text{Scenarios}} 
\sum_{c \in \text{Class}} \sum_{h \in \text{Options}}
{\text{prob}_i * \text{prob}_j * \text{prob}_k * \text{price3}_{c,h} * p2_{i,j,c,h} * s3_{i,j,k,c,h}} )
- \text{cost} * n
</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Constraints">¶</a></h3><p><strong>Price option week 1</strong>: Only one price option must be chosen in each class in week 1.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-67-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-635" style="width: 15.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 12.979em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1012.92em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-636"><span class="munderover" id="MathJax-Span-637"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-638" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-639"><span class="mrow" id="MathJax-Span-640"><span class="mi" id="MathJax-Span-641" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-642" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-643" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mi" id="MathJax-Span-644" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="msubsup" id="MathJax-Span-645"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-646" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-647"><span class="mrow" id="MathJax-Span-648"><span class="mi" id="MathJax-Span-649" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-650" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-651" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-652" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-653" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-654" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-655" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-656" style="font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-657" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-658" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mi>p</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>=</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>c</mi><mo>∈</mo><mtext>Class</mtext></math></span></span></div><script id="MathJax-Element-67" type="math/tex; mode=display">
\sum_{h \in \text{Options}} p1_{c,h} = 1 \quad \forall c \in \text{Class}
</script><p><strong>Sales week 1</strong>: Sales cannot exceed forecasted demand in week 1.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;forecast1&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-68-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-659" style="width: 36.67em; display: inline-block;"><span style="display: inline-block; position: relative; width: 30.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1030.48em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-660"><span class="mi" id="MathJax-Span-661" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-662"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-663" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-664"><span class="mrow" id="MathJax-Span-665"><span class="mi" id="MathJax-Span-666" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-667" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-668" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-669" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-670" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-671" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-672" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.81em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-673" style="font-family: MathJax_Main;">forecast1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.872em;"><span class="texatom" id="MathJax-Span-674"><span class="mrow" id="MathJax-Span-675"><span class="mi" id="MathJax-Span-676" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-677" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-678" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-679" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-680" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-681" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-682" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">p</span><span class="msubsup" id="MathJax-Span-683"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-684" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-685"><span class="mrow" id="MathJax-Span-686"><span class="mi" id="MathJax-Span-687" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-688" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-689" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-690" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-691" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-692" style="font-family: MathJax_Main; padding-left: 0.182em;">∀</span><span class="mi" id="MathJax-Span-693" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-694" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-695" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span><span class="mo" id="MathJax-Span-696" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-697" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-698" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">c</span><span class="mo" id="MathJax-Span-699" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-700" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span><span class="mo" id="MathJax-Span-701" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-702" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-703" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">h</span><span class="mo" id="MathJax-Span-704" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-705" style="font-family: MathJax_Main; padding-left: 0.301em;">Options</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>≤</mo><msub><mtext>forecast1</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>p</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>,</mo><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><mtext>Scenarios</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>c</mi><mo>∈</mo><mtext>Class</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>h</mi><mo>∈</mo><mtext>Options</mtext></math></span></span></div><script id="MathJax-Element-68" type="math/tex; mode=display">
s1_{i,c,h} \leq \text{forecast1}_{i,c,h} * p1_{c,h},
\quad \forall i \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
</script><p><strong>Price option week 2</strong>: Only one price option must be chosen in each class in week 2 for each scenario in week 1.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-69-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-706" style="width: 23.813em; display: inline-block;"><span style="display: inline-block; position: relative; width: 19.824em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1019.76em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-707"><span class="munderover" id="MathJax-Span-708"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-709" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-710"><span class="mrow" id="MathJax-Span-711"><span class="mi" id="MathJax-Span-712" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-713" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-714" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mi" id="MathJax-Span-715" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="msubsup" id="MathJax-Span-716"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-717" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-718"><span class="mrow" id="MathJax-Span-719"><span class="mi" id="MathJax-Span-720" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-721" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-722" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-723" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-724" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-725" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-726" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-727" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-728" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-729" style="font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-730" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-731" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span><span class="mo" id="MathJax-Span-732" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-733" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-734" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">i</span><span class="mo" id="MathJax-Span-735" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-736" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mi>p</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>=</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>c</mi><mo>∈</mo><mtext>Class</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>i</mi><mo>∈</mo><mtext>Scenarios</mtext></math></span></span></div><script id="MathJax-Element-69" type="math/tex; mode=display">
\sum_{h \in \text{Options}} p2_{i,c,h} = 1 \quad \forall c \in \text{Class}, \; i \in \text{Scenarios}
</script><p><strong>Sales week 2</strong>: Sales cannot exceed forecasted demand in week 2.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;forecast2&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-70-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-737" style="width: 38.932em; display: inline-block;"><span style="display: inline-block; position: relative; width: 32.443em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1032.38em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-738"><span class="mi" id="MathJax-Span-739" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-740"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-741" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-742"><span class="mrow" id="MathJax-Span-743"><span class="mi" id="MathJax-Span-744" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-745" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-746" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-747" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-748" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-749" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-750" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-751" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-752" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.36em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.81em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-753" style="font-family: MathJax_Main;">forecast2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.872em;"><span class="texatom" id="MathJax-Span-754"><span class="mrow" id="MathJax-Span-755"><span class="mi" id="MathJax-Span-756" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-757" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-758" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-759" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-760" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-761" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-762" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">p</span><span class="msubsup" id="MathJax-Span-763"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-764" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-765"><span class="mrow" id="MathJax-Span-766"><span class="mi" id="MathJax-Span-767" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-768" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-769" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-770" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-771" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-772" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-773" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-774" style="font-family: MathJax_Main; padding-left: 0.182em;">∀</span><span class="mi" id="MathJax-Span-775" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-776" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-777" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-778" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-779" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span><span class="mo" id="MathJax-Span-780" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-781" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-782" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">c</span><span class="mo" id="MathJax-Span-783" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-784" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span><span class="mo" id="MathJax-Span-785" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-786" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-787" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">h</span><span class="mo" id="MathJax-Span-788" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-789" style="font-family: MathJax_Main; padding-left: 0.301em;">Options</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>≤</mo><msub><mtext>forecast2</mtext><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>p</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>,</mo><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>,</mo><mi>j</mi><mo>∈</mo><mtext>Scenarios</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>c</mi><mo>∈</mo><mtext>Class</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>h</mi><mo>∈</mo><mtext>Options</mtext></math></span></span></div><script id="MathJax-Element-70" type="math/tex; mode=display">
s2_{i,j,c,h} \leq \text{forecast2}_{j,c,h} * p2_{i,c,h},
\quad \forall i,j \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
</script><p><strong>Price option week 3</strong>: Only one price option must be chosen in each class in week 3 for each scenario in week 1 and week 2.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-71-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-790" style="width: 25.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 21.193em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1021.13em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-791"><span class="munderover" id="MathJax-Span-792"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-793" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-794"><span class="mrow" id="MathJax-Span-795"><span class="mi" id="MathJax-Span-796" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-797" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-798" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mi" id="MathJax-Span-799" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="msubsup" id="MathJax-Span-800"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-801" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-802"><span class="mrow" id="MathJax-Span-803"><span class="mi" id="MathJax-Span-804" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-805" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-806" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-807" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-808" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-809" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-810" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-811" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-812" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-813" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-814" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-815" style="font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-816" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-817" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span><span class="mo" id="MathJax-Span-818" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-819" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-820" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">i</span><span class="mo" id="MathJax-Span-821" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-822" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-823" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-824" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mi>p</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>=</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>c</mi><mo>∈</mo><mtext>Class</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>i</mi><mo>,</mo><mi>j</mi><mo>∈</mo><mtext>Scenarios</mtext></math></span></span></div><script id="MathJax-Element-71" type="math/tex; mode=display">
\sum_{h \in \text{Options}} p3_{i,j,c,h} = 1 \quad \forall c \in \text{Class}, \; i,j \in \text{Scenarios}
</script><p><strong>Sales week 3</strong>: Sales cannot exceed forecasted demand in week 3.
<span class="MathJax_Preview" style="color: inherit; display: none;"></span></p><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;forecast3&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-72-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-825" style="width: 41.432em; display: inline-block;"><span style="display: inline-block; position: relative; width: 34.527em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1034.47em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-826"><span class="mi" id="MathJax-Span-827" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-828"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-829" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-830"><span class="mrow" id="MathJax-Span-831"><span class="mi" id="MathJax-Span-832" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-833" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-834" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-835" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-836" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-837" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-838" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-839" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-840" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-841" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-842" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.42em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.81em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-843" style="font-family: MathJax_Main;">forecast3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.872em;"><span class="texatom" id="MathJax-Span-844"><span class="mrow" id="MathJax-Span-845"><span class="mi" id="MathJax-Span-846" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-847" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-848" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-849" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-850" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-851" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-852" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">p</span><span class="msubsup" id="MathJax-Span-853"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-854" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-855"><span class="mrow" id="MathJax-Span-856"><span class="mi" id="MathJax-Span-857" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-858" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-859" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-860" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-861" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-862" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-863" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-864" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-865" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-866" style="font-family: MathJax_Main; padding-left: 0.182em;">∀</span><span class="mi" id="MathJax-Span-867" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-868" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-869" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-870" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-871" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-872" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-873" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span><span class="mo" id="MathJax-Span-874" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-875" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-876" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">c</span><span class="mo" id="MathJax-Span-877" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-878" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span><span class="mo" id="MathJax-Span-879" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-880" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-881" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">h</span><span class="mo" id="MathJax-Span-882" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-883" style="font-family: MathJax_Main; padding-left: 0.301em;">Options</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>≤</mo><msub><mtext>forecast3</mtext><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>∗</mo><mi>p</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>,</mo><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>∈</mo><mtext>Scenarios</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>c</mi><mo>∈</mo><mtext>Class</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>h</mi><mo>∈</mo><mtext>Options</mtext></math></span></span></div><script id="MathJax-Element-72" type="math/tex; mode=display">
s3_{i,j,k,c,h} \leq \text{forecast3}_{k,c,h} * p3_{i,j,c,h},
\quad \forall i,j,k \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
</script>
<p><strong>Class capacity</strong>: Capacity constraint for each class.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Options&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msub&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;cap&lt;/mtext&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Scenarios&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mspace width="thickmathspace" /&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Class&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-73-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-884" style="width: 48.693em; display: inline-block;"><span style="display: inline-block; position: relative; width: 40.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1040.48em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-885"><span class="munderover" id="MathJax-Span-886"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-887" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-888"><span class="mrow" id="MathJax-Span-889"><span class="mi" id="MathJax-Span-890" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-891" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-892" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mi" id="MathJax-Span-893" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">s</span><span class="msubsup" id="MathJax-Span-894"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-895" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-896"><span class="mrow" id="MathJax-Span-897"><span class="mi" id="MathJax-Span-898" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-899" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-900" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-901" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-902" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-903" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-904" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-905" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-906"><span class="mrow" id="MathJax-Span-907"><span class="mi" id="MathJax-Span-908" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-909" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-910" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mi" id="MathJax-Span-911" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">s</span><span class="msubsup" id="MathJax-Span-912"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-913" style="font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-914"><span class="mrow" id="MathJax-Span-915"><span class="mi" id="MathJax-Span-916" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-917" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-918" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-919" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-920" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-921" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-922" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-923" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-924" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.955em;"><span class="mo" id="MathJax-Span-925" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.28em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-926"><span class="mrow" id="MathJax-Span-927"><span class="mi" id="MathJax-Span-928" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-929" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-930" style="font-size: 70.7%; font-family: MathJax_Main;">Options</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mi" id="MathJax-Span-931" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">s</span><span class="msubsup" id="MathJax-Span-932"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-933" style="font-family: MathJax_Main;">3</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-934"><span class="mrow" id="MathJax-Span-935"><span class="mi" id="MathJax-Span-936" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-937" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-938" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-939" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-940" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-941" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-942" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-943" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-944" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-945" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-946" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.908em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-947" style="font-family: MathJax_Main;">cap</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 1.491em;"><span class="mi" id="MathJax-Span-948" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-949" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mi" id="MathJax-Span-950" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">n</span><span class="mspace" id="MathJax-Span-951" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-952" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-953" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-954" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-955" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-956" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-957" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-958" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-959" style="font-family: MathJax_Main; padding-left: 0.301em;">Scenarios</span><span class="mo" id="MathJax-Span-960" style="font-family: MathJax_Main;">,</span><span class="mspace" id="MathJax-Span-961" style="height: 0em; vertical-align: 0em; width: 0.301em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-962" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">c</span><span class="mo" id="MathJax-Span-963" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-964" style="font-family: MathJax_Main; padding-left: 0.301em;">Class</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mi>s</mi><msub><mn>1</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mi>s</mi><msub><mn>2</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>Options</mtext></mrow></munder><mi>s</mi><msub><mn>3</mn><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>,</mo><mi>c</mi><mo>,</mo><mi>h</mi></mrow></msub><mo>≤</mo><msub><mtext>cap</mtext><mi>c</mi></msub><mo>∗</mo><mi>n</mi><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi><mo>∈</mo><mtext>Scenarios</mtext><mo>,</mo><mspace width="thickmathspace"></mspace><mi>c</mi><mo>∈</mo><mtext>Class</mtext></math></span></span></div><script id="MathJax-Element-73" type="math/tex; mode=display">
\sum_{h \in \text{Options}} s1_{i,c,h} +
\sum_{h \in \text{Options}} s2_{i,j,c,h} + 
\sum_{h \in \text{Options}} s3_{i,j,k,c,h} \leq \text{cap}_c * n
\quad \forall i,j,k \in \text{Scenarios}, \; c \in \text{Class}
</script><p><strong>Planes</strong> Up to six planes can be hired.</p>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mn&gt;6&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-74-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-965" style="width: 3.039em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.44em, 2.503em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-966"><span class="mi" id="MathJax-Span-967" style="font-family: MathJax_Math-italic;">n</span><span class="mo" id="MathJax-Span-968" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mn" id="MathJax-Span-969" style="font-family: MathJax_Main; padding-left: 0.301em;">6</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.211em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mi>n</mi><mo>≤</mo><mn>6</mn></math></span></span></div><script id="MathJax-Element-74" type="math/tex; mode=display">
n \leq 6
</script>



<hr/>
<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Python-Implementation">¶</a></h2><p>We import the Gurobi Python Module and other Python libraries.</p>


In [ ]:

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0




<h2 id="Input-Data">Input Data<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Input-Data">¶</a></h2><p>We define all the input data for the model.</p>


In [ ]:

# Lists of classes, price options, and scenarios.

classes = ['First', 'Business', 'Economy']

options = ['option1', 'option2', 'option3']

scenarios = ['sce1', 'sce2', 'sce3']

#  Classes, price options, and prices value for week 1, 2, and 3.

ch, price1, price2, price3  = gp.multidict({
    ('First', 'option1'): [1200, 1400, 1500],
    ('Business', 'option1'): [900, 1100, 820],
    ('Economy', 'option1'): [500, 700, 480],
    ('First', 'option2'): [1000, 1300, 900],
    ('Business', 'option2'): [800, 900, 800],
    ('Economy', 'option2'): [300, 400, 470],
    ('First', 'option3'): [950, 1150, 850],
    ('Business', 'option3'): [600, 750, 500],
    ('Economy', 'option3'): [200, 350, 450]
})

# Probablity of each scenario

prob ={'sce1': 0.1, 'sce2': 0.7, 'sce3': 0.2}

# Forecasted demand for each class, price option, and scenario at week 1

ich, fcst1, fcst2, fcst3 = gp.multidict({
    ('sce1', 'First', 'option1'): [10, 20, 30],
    ('sce1', 'Business', 'option1'): [20, 42, 40],
    ('sce1', 'Economy', 'option1'): [45, 50, 50],
    ('sce1', 'First', 'option2'): [15, 25, 35],
    ('sce1', 'Business', 'option2'): [25, 45, 50],
    ('sce1', 'Economy', 'option2'): [55, 52, 60],
    ('sce1', 'First', 'option3'): [20, 35, 40],
    ('sce1', 'Business', 'option3'): [35, 46, 55],
    ('sce1', 'Economy', 'option3'): [60, 60, 80],
    ('sce2', 'First', 'option1'): [20, 10, 30],
    ('sce2', 'Business', 'option1'): [40, 50, 10],
    ('sce2', 'Economy', 'option1'): [50, 60, 50],
    ('sce2', 'First', 'option2'): [25, 40, 40],
    ('sce2', 'Business', 'option2'): [42, 60, 40],
    ('sce2', 'Economy', 'option2'): [52, 65, 60],
    ('sce2', 'First', 'option3'): [35, 50, 60],
    ('sce2', 'Business', 'option3'): [45, 80, 45],
    ('sce2', 'Economy', 'option3'): [63, 90, 70],
    ('sce3', 'First', 'option1'): [45, 50, 50],
    ('sce3', 'Business', 'option1'): [45, 20, 40],
    ('sce3', 'Economy', 'option1'): [55, 10, 60],
    ('sce3', 'First', 'option2'): [50, 55, 70],
    ('sce3', 'Business', 'option2'): [46, 30, 45],
    ('sce3', 'Economy', 'option2'): [56, 40, 65],
    ('sce3', 'First', 'option3'): [60, 80, 80],
    ('sce3', 'Business', 'option3'): [47, 50, 60],
    ('sce3', 'Economy', 'option3'): [64, 60, 70]    
})

#  Actual demand at weeks 1, 2, and 3.

ch, demand1, demand2, demand3  = gp.multidict({
    ('First', 'option1'): [25, 22, 45],
    ('Business', 'option1'): [50, 45, 20],
    ('Economy', 'option1'): [50, 50, 55],
    ('First', 'option2'): [30, 45, 60],
    ('Business', 'option2'): [40, 55, 40],
    ('Economy', 'option2'): [53, 60, 60],
    ('First', 'option3'): [40, 50, 75],
    ('Business', 'option3'): [45, 75, 50],
    ('Economy', 'option3'): [65, 80, 75]
})

# Class capacity

cap ={'First': 37, 'Business': 38, 'Economy': 47}

# cost per plane

cost = 50000



In [ ]:

# Preprocessing

# week 1 data structures
list_ch = []

for c in classes:
    for h in options:
        tp = c,h
        list_ch.append(tp)

ch = gp.tuplelist(list_ch)

list_ich = []

for i in scenarios:
    for c in classes:
        for h in options:
            tp = i,c,h
            list_ich.append(tp)

ich = gp.tuplelist(list_ich)

# week 2 data structure

list_ijch = []

for i in scenarios:
    for j in scenarios:
        for c in classes:
            for h in options:
                tp = i,j,c,h
                list_ijch.append(tp)

ijch = gp.tuplelist(list_ijch)

# week 3 data structure

list_ijkch = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            for c in classes:
                for h in options:
                    tp = i,j,k,c,h
                    list_ijkch.append(tp)

ijkch = gp.tuplelist(list_ijkch)

# scenarios data structure

list_ijk = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            tp = i,j,k,
            list_ijk.append(tp)

ijk = gp.tuplelist(list_ijk)

# capacity constraints data structure

list_ijkc = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            for c in classes:
                tp = i,j,k,c
                list_ijkc.append(tp)

ijkc = gp.tuplelist(list_ijkc)




<h2 id="Model-Deployment">Model Deployment<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Model-Deployment">¶</a></h2><p>Solving bilinear problems with Gurobi is as easy as configuring the global parameter <code>nonConvex</code>, and setting this parameter to the value of 2.</p>
<h3 id="First-Period-Model">First Period Model<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#First-Period-Model">¶</a></h3><p>At the beginning of week 1, we want to determine the price options for this week.</p>
<p>We create a model and the variables.</p>


In [ ]:

model = gp.Model('YieldManagement')

# Set global parameters. 
model.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model.addVars(ch, vtype=GRB.BINARY, name="p1ch")
p2ich = model.addVars(ich, vtype=GRB.BINARY, name="p2ich")
p3ijch = model.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# tickets to be sold at each week
s1ich = model.addVars(ich, name="s1ich")
s2ijch = model.addVars(ijch, name="s2ijch")
s3ijkch = model.addVars(ijkch, name="s3ijkch")

# number of planes to fly
n = model.addVar(ub=6, vtype=GRB.INTEGER, name="planes")




<h3 id="Week-1-constraints">Week 1 constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Week-1-constraints">¶</a></h3><p>The following constraints ensure that only one price option is chosen in each class in week 1.</p>


In [ ]:

# Price option constraints for week 1

priceOption1 = model.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )




<p>The following constraints enforce that sales cannot exceed forecasted demand in week 1.</p>


In [ ]:

# sales constraints for week 1

sales1 = model.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )




<h3 id="Week-2-constraints">Week 2 constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Week-2-constraints">¶</a></h3><p>Only one price option must be chosen in each class in week 2 for each scenario in week 1.</p>


In [ ]:

# Price option constraints for week 2

priceOption2 = model.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )




<p>Sales cannot exceed forecasted demand in week 2.</p>


In [ ]:

# sales constraints for week 2

sales2 = model.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )




<h3 id="Week-3-constraints">Week 3 constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Week-3-constraints">¶</a></h3><p>Only one price option must be chosen in each class in week 3 for each scenario in week 1 and week 2.</p>


In [ ]:

# Price option constraints for week 3

priceOption3 = model.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )




<p>Sales cannot exceed forecasted demand in week 3.</p>


In [ ]:

# sales constraints for week 3

sales3 = model.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )




<p>Capacity constraint for each class.</p>


In [ ]:

# Class capacity constraints

classCap = model.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')




<p>The objective is to maximize expected profit.</p>


In [ ]:

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model.setObjective( obj, GRB.MAXIMIZE)



In [ ]:

# Verify model formulation

model.write('YieldManagement.lp')

# Run optimization engine

model.optimize()

#############################################################
#            Print results of model for week 1
#############################################################

print("\n\n\n____________________Week 1 solution___________________________")

print(f"The expected total profit is: £{round(model.objVal,2): ,}") 
print(f"Number of planes to book: {n.x}")

# Week 1 prices for seat class

# optimal values of option prices at week 1
opt_p1ch = {}

print("\n____________________Week 1 prices_______________________________")
for c,h in ch:
    opt_p1ch[c,h] = 0
    if p1ch[c,h].x > 0.5:
        opt_p1ch[c,h] = round(p1ch[c,h].x)
        price_ch = opt_p1ch[c,h]*price1[c,h]
        print(f"({c},{h}) = £{price_ch: ,}")
#

# Week 2 provisional prices

print("\n_____________Week 2 provisional prices____________________________")
for i,c,h in ich:
    if p2ich[i,c,h].x > 0.5:
        price_ch = round(p2ich[i,c,h].x)*price2[c,h]
        print(f"({i}, {c}, {h}) = £{price_ch: ,}")

# Week 3 provisional prices

print("\n_____________Week 3 provisional prices____________________________")
for i,j,c,h in ijch:
    if p3ijch[i,j,c,h].x > 0.5:
        price_ch = round(p3ijch[i,j,c,h].x)*price3[c,h]
        print(f"({i}, {j}, {c}, {h}) = £{price_ch: ,}")




<h3 id="Second-Period-model">Second Period model<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Second-Period-model">¶</a></h3><p>We ran the model using the actual demand of week 1 and determine the price options for week 2.</p>


In [ ]:

model2 = gp.Model('YieldManagement2')

# Set global parameters 
model2.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model2.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# Fix price options of week 1
for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model2.addVars(ich, vtype=GRB.BINARY, name="p2ich")
p3ijch = model2.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# tickets to be sold at each week
s1ich = model2.addVars(ich, name="s1ich")

# use hindsight demand of week 1
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h] 

s2ijch = model2.addVars(ijch, name="s2ijch")
s3ijkch = model2.addVars(ijkch, name="s3ijkch")

# number of planes to fly
n = model2.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# Price option constraints for week 1

priceOption1 = model2.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# sales constraints for week 1

sales1 = model2.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# Price option constraints for week 2

priceOption2 = model2.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# sales constraints for week 2

sales2 = model2.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# Price option constraints for week 3

priceOption3 = model2.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# sales constraints for week 3

sales3 = model2.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# Class capacity constraints.

classCap = model2.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model2.setObjective( obj, GRB.MAXIMIZE)

# Verify model formulation

model2.write('YieldManagement2.lp')

# Run optimization engine

model2.optimize()

#############################################################
#            Print results of model for week 2
#############################################################

print("\n\n\n____________________Week 2 solution___________________________")

print(f"The expected total profit at the beginning of week 2 is: £ {round(model2.objVal,2):,}") 
print(f"Number of planes to book: {n.x}")

# Week 2 prices

# optimal values of option prices at week 1
opt_p2ich = {}

print("\n_____________Week 2 prices____________________________")
for i,c,h in ich:
    opt_p2ich[i,c,h] = 0
    if p2ich[i,c,h].x > 0.5:
        opt_p2ich[i,c,h] = round(p2ich[i,c,h].x)
        price_ch = opt_p2ich[i,c,h]*price2[c,h]
        #print(f"({i},{c},{h}) = £ {price_ch}")
        if i == 'sce1':
            print(f"({c},{h}) = £{price_ch: ,}")
#

# Week 3 provisional prices

print("\n_____________Week 3 provisional prices____________________________")
for i,j,c,h in ijch:
    if p3ijch[i,j,c,h].x > 0.5:
        price_ch = round(p3ijch[i,j,c,h].x)*price3[c,h]
        print(f"({i}, {j}, {c}, {h}) = £ {price_ch}")




<h1 id="Third-Period-model">Third Period model<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Third-Period-model">¶</a></h1><p>We ran the model using the actual demand of weeks 1 and 2.</p>


In [ ]:

model3 = gp.Model('YieldManagement3')

# Set global parameters. 
model3.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model3.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# Fix price options of week 1

for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model3.addVars(ich, vtype=GRB.BINARY, name="p2ich")

# Fix price options of week 2

for i,c,h in ich:
    p2ich[i,c,h].lb = opt_p2ich[i,c,h]

p3ijch = model3.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# tickets to be sold at each week
s1ich = model3.addVars(ich, name="s1ich")

# use hindsight demand of week 1
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h]

s2ijch = model3.addVars(ijch, name="s2ijch")

# use hindsight demand of week 2
for j,c,h in ich:
    fcst2[j,c,h] = 0
    fcst2[j,c,h] = demand2[c,h]*opt_p2ich[j,c,h]

s3ijkch = model3.addVars(ijkch, name="s3ijkch")

# number of planes to fly
n = model3.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# Price option constraints for week 1

priceOption1 = model3.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# sales constraints for week 1

sales1 = model3.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# Price option constraints for week 2

priceOption2 = model3.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# sales constraints for week 2

sales2 = model3.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# Price option constraints for week 3

priceOption3 = model3.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# sales constraints for week 3

sales3 = model3.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# Class capacity constraints.

classCap = model3.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model3.setObjective( obj, GRB.MAXIMIZE)

# Verify model formulation

model3.write('YieldManagement3.lp')

# Run optimization engine

model3.optimize()

#############################################################
#            Print results of model for week 3
#############################################################


print("\n\n\n____________________Week 3 solution___________________________")

print(f"The expected total profit is: £ {round(model3.objVal,2):,}") 
print(f"Number of planes to book: {n.x}")

# Week 3  prices

# optimal values of option prices at week 3
opt_p3ijch = {}


print("\n_____________Week 3 prices____________________________")
for i,j,c,h in ijch:
    opt_p3ijch[i,j,c,h] = 0
    if p3ijch[i,j,c,h].x > 0.5:
        opt_p3ijch[i,j,c,h] = round(p3ijch[i,j,c,h].x)
        price_ch = opt_p3ijch[i,j,c,h]*price3[c,h]
        if i == 'sce1' and j == 'sce1':
            print(f"({c}, {h}) = £{price_ch: ,}")




<h1 id="Solution-at-take-off">Solution at take-off<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#Solution-at-take-off">¶</a></h1><p>We ran the model using the actual demand of weeks 1, 2, and 3.</p>


In [ ]:

model4 = gp.Model('YieldManagement4')

# Set global parameters 
model4.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model4.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# Fix price options of week 1

for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model4.addVars(ich, vtype=GRB.BINARY, name="p2ich")

# Fix price options of week 2

for i,c,h in ich:
    p2ich[i,c,h].lb = opt_p2ich[i,c,h]

p3ijch = model4.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# Capture one scenario where opt_p3ijch[i,j,c,h] = 1
opt_p3kch = {}

for i,j,c,h in ijch:
    p3ijch[i,j,c,h].lb = opt_p3ijch[i,j,c,h]
    opt_p3kch[j,c,h] = 0
    if opt_p3ijch[i,j,c,h] == 1:
        opt_p3kch[j,c,h] = opt_p3ijch[i,j,c,h] 

# tickets to be sold at each week
s1ich = model4.addVars(ich, name="s1ich")

# use hindsight demand of week 1
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h] 

s2ijch = model4.addVars(ijch, name="s2ijch")

# use hindsight demand of week 2
for j,c,h in ich:
    fcst2[j,c,h] = 0
    fcst2[j,c,h] = demand2[c,h]*opt_p2ich[j,c,h]


s3ijkch = model4.addVars(ijkch, name="s3ijkch")

# use hindsight demand of week 3
for k,c,h in ich:
    fcst3[k,c,h] = 0
    fcst3[k,c,h] = demand3[c,h]*opt_p3kch[k,c,h]

    
# number of planes to fly
n = model4.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# Price option constraints for week 1

priceOption1 = model4.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# sales constraints for week 1

sales1 = model4.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# Price option constraints for week 2

priceOption2 = model4.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# sales constraints for week 2

sales2 = model4.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# Price option constraints for week 3

priceOption3 = model4.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# sales constraints for week 3

sales3 = model4.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# Class capacity constraints.

classCap = model4.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model4.setObjective( obj, GRB.MAXIMIZE)

# Verify model formulation

model4.write('YieldManagement4.lp')

# Run optimization engine

model4.optimize()

#############################################################
#            Print results of model for week 4
#############################################################


print("\n\n\n____________________Take off solution___________________________")

print(f"The actual total profit is: £{round(model4.objVal,2):,}") 
print(f"Number of planes used: {n.x}")


# Sales week 1

print("\n___________Week 1 seats sold and revenue__________________________")
for i,c,h in ich:
    if i == 'sce1':
        if s1ich[i,c,h].x > 1e-6:
            tickets = round(s1ich[i,c,h].x)
            price = price1[c,h]*round(p1ch[c,h].x)
            revenue = price*tickets
            print(f"{c} class: {tickets} seats sold at £{price:,}: revenue £{revenue:,}  ")
            
# Sales week 2

print("___________Period 2 seats sold and revenue__________________________")
for i,j,c,h in ijch:
    if i == 'sce1' and j == 'sce1':
        if s2ijch[i,j,c,h].x > 1e-6:
            tickets = round(s2ijch[i,j,c,h].x)
            price = price2[c,h]*round(p2ich[i,c,h].x)
            revenue = price*tickets
            print(f"{c} class: {tickets} seats sold at £{price:,}: revenue £{revenue:,}  ")
            
# Sales week 3

print("___________Period 3 seats sold and revenue__________________________")
for i,j,k,c,h in ijkch:
    if i == 'sce1' and j == 'sce1' and k == 'sce1':
        if s3ijkch[i,j,k,c,h].x > 1e-6:
            tickets = round(s3ijkch[i,j,k,c,h].x )
            price = price3[c,h]*round(p3ijch[i,j,c,h].x)
            revenue = price*tickets
            print(f"{c} class: {tickets} seats sold at £{price:,}: revenue £{revenue:,}  ")




<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/yield_management/yield_management.html#References">¶</a></h2><p>H. Paul Williams, Model Building in Mathematical Programming, fifth edition.</p>
<p>Copyright © 2020 Gurobi Optimization, LLC</p>
